In [ ]:
import time
import pandas as pd
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore

In [ ]:
cred = credentials.Certificate('../../src/config/argoai-63051-firebase-adminsdk-ykwbi-a14bbb8c68.json')

app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [ ]:
bulk_writer = db.bulk_writer()
batch = db.batch()
collection = db.collection('mrv_system')

In [ ]:
location_data = ['latitude', 'longitude', 'district', 'province', 'address']
weather_data = ['max_temperature', 'min_temperature', 'wind', 'wind_direction', 'humidity', 'cloud',
                'atmospheric_pressure']
soil_data = ['soil_moisture', 'soil_moisture_20', 'soil_moisture_40', 'soil_temperature', 'soil_ph',
             'soil_conductivity']
irrigation_data = ['water_consumed', 'water_recycled']
pest_data = ['pest_population_counts', 'disease_incidence', 'severity_of_infestations']


In [ ]:
def create_data_item(row):
    return {
        'fertilize': row.get('fertilize', None),
        'weather': {key: row.get(key, None) for key in weather_data},
        'pest': {key: row.get(key, None) for key in pest_data},
        'irrigation': {key: row.get(key, None) for key in irrigation_data},
        'soil': {key: row.get(key, None) for key in soil_data},
    }

In [ ]:
def write_data_to_firestore(df):
    bulk_writer = db.bulk_writer()

    for index, row in df.iterrows():
        print(f"index: {index}")
        data_item = create_data_item(row)
        facility = row.get('facility', None)
        doc_ref = db.collection("mrv_system").document()
        data = {
            'plant': str(row['plant']).lower().strip(),
            'date': datetime.strptime(
                datetime.strptime(row["datetime"].strftime("%d/%m/%Y"), "%d/%m/%Y").strftime("%Y/%m/%d"), "%Y/%m/%d"),
            'facility': str(facility).lower(),
            'location': {key: row.get(key, None) for key in location_data},
            'data': data_item
        }
        print(f"data: {data}")
        print("________")
        bulk_writer.set(doc_ref, data)

    bulk_writer.flush()

In [ ]:
df = pd.read_excel('../data/data.xlsx', sheet_name='data_temp')

In [ ]:
start = time.time()
write_data_to_firestore(df)
end = time.time()
print(f'Execution time: {end - start} seconds')